In [ ]:
import requests
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
import shutil
from PIL import Image

from sklearn.model_selection import train_test_split

In [ ]:
X, y = make_classification(n_samples=1000, n_features=4, n_classes=2, n_informative=2, random_state=42)
X_multi, y_multi = make_classification(n_samples=500, n_features=6, n_classes=3, n_informative=6,
                                      n_redundant=0, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Xm_train, Xm_test, ym_train, ym_test = train_test_split(X_multi, y_multi, test_size=0.2, random_state=42,
                                                       stratify=y_multi)


X_train, X_test, y_train, y_test = pd.DataFrame(X_train).to_dict(), pd.DataFrame(X_test).to_dict(), \
                                   pd.DataFrame(y_train).to_dict(), pd.DataFrame(y_test).to_dict()

Xm_train, Xm_test, ym_train, ym_test = pd.DataFrame(Xm_train).to_dict(), pd.DataFrame(Xm_test).to_dict(), \
                                       pd.DataFrame(ym_train).to_dict(), pd.DataFrame(ym_test).to_dict()

# 0. Взаимодействие с БД

In [ ]:
print(requests.put('http://127.0.0.1:5000/api/add_row/1', json={'model': 'world'}).text)

In [ ]:
print(requests.post('http://127.0.0.1:5000/api/update_row/1', json={'model': 'Fitted'}).text)

In [ ]:
print(requests.get('http://127.0.0.1:5000/api/get_row/1').text)

In [ ]:
print(requests.delete('http://127.0.0.1:5000/api/delete_row/1').text)

# 1. Доступные типы моделей:

- 127.0.0.1:5000
- 80.249.151.221:5000

In [ ]:
print(requests.get('http://80.249.151.221:5000/api/model_types').text)

In [ ]:
# print(requests.get('http://127.0.0.1:5000/api/results', json={'task_id': '090ebd70-f544-430f-82d5-10f27d5b2601'}).text)

# 2. Создание моделей

### 2.1 Корректный запрос, но с ошибкой в переменной

In [ ]:
print(requests.post('http://80.249.151.221:5000/api/create_model', json={
    'model_type': 'LR',
    'dataset_name': 'test'
}).text)

In [ ]:
print(requests.post('http://127.0.0.1:5000/api/create_model', json={
    'model_type': 'LR',
    'model_name': 'LR',
    'dataset_name': 'binary'
}).text)

print(requests.post('http://127.0.0.1:5000/api/create_model', json={
    'model_type': 'boosting',
    'dataset_name': 'multiclass'
}).text)

print(requests.post('http://127.0.0.1:5000/api/create_model', json={
    'model_name': 'fit_test',
    'model_type': 'LogisticRegression (LR)'
}).text)

### 2.2 Некорректный запрос

In [ ]:
print(requests.post('http://127.0.0.1:5000/api/create_model', json={
    'model_type': 'Tree',
    'model_name': 'test_name',
    'dataset_name': 'test_name'
}).text)

# 3. Принт доступных моделей

### 3.0 Все модели

In [ ]:
r = requests.get('http://80.249.151.221:5000/api/get_model')
try: 
    print(r.json())
except:
    print(r.text)

### 3.1 Несуществующая модель

In [ ]:
r = requests.get('http://127.0.0.1:5000/api/get_model/20')
try: 
    print(r.json())
except:
    print(r.text)

### 3.2. Правильный id модели

In [ ]:
print(requests.get('http://127.0.0.1:5000/api/get_model/2').text)

# 4. Обновление модели

### 4.1. Вызов без словаря

In [ ]:
print(requests.post('http://127.0.0.1:5000/api/update_model').text)

### 4.2. Вызов с некорректным словарем

In [ ]:
print(requests.post('http://127.0.0.1:5000/api/update_model', json={
    'model_type': 'test',
    'dataset_name': 'multi_class'
}).text)

### 4.3 Корректный вызов

In [ ]:
print(requests.post('http://127.0.0.1:5000/api/update_model', json={
    'model_id': 1,
    'model_type': 'test',
    'dataset_name': 'test_2'
}).text)

# 5. Удаление модели

### 5.1 Correct

In [ ]:
print(requests.delete('http://80.249.151.221:5000/api/delete_model/1').text)

### 5.2 Incorrect

In [ ]:
print(requests.delete('http://127.0.0.1:5000/api/delete_model/500').text)

# 6. Установление гиперпараметров вручную

### 6.1 Корректные гиперпараметры

In [ ]:
print(requests.put('http://127.0.0.1:5000/api/set_model_params/2', json={'C': 2}).text)

### 6.2 Некорректные гиперпараметры

In [ ]:
print(requests.put('http://127.0.0.1:5000/api/set_model_params/2', json={'abc': -1}).text)

# 7. Доступные для оптимазации гиперпараметры

### 7.1 Корректный тип модели

In [ ]:
print(requests.get('http://127.0.0.1:5000/api/available_opt_params', json={'model_type': 'LR'}).text)

### 7.2 Некорректный тип модели

In [ ]:
print(requests.get('http://127.0.0.1:5000/api/available_opt_params', json={'model_type': 'Tree'}).text)

### 7.3 Ввод без словаря

In [ ]:
print(requests.get('http://127.0.0.1:5000/api/available_opt_params').text)

# 8. Оптимизация гиперпараметров

### 8.1 Правильная передача гиперпараметров

In [ ]:
params_to_optimaze = {'C': [0, 10],
                      'tol': [0.000001, 0.01],
                      'max_iter': [10, 1000]}

json = {'X': X_train,
        'y': y_train,
        'metric': 'auc_roc',
        'params_to_optimaze':{}}

print(requests.get('http://80.249.151.221:5000/api/optimize_model_params/1', json=json).text)

In [ ]:
print(requests.get('http://80.249.151.221:5000/api/results', json={'task_id': 'ded0c4aa-add4-482d-95c1-cad98023c619'}).text)

In [ ]:
params_to_optimaze = {
'boosting_type': ['goss'],
'n_estimators': [10, 100],
'learning_rate': [0.1, 0.2], 
'num_leaves': [10, 100],
'max_depth': [1, 10],  
'min_child_weight': [2, 100], 
'reg_alpha': [0.1, 0.2], 
'reg_lambda': [0.1, 0.2]
}

json = {'X': Xm_train,
        'y': ym_train,
        'metric': 'auc_pr',
        'params_to_optimaze': params_to_optimaze,
       'max_trails': 100}

print(requests.get('http://127.0.0.1:5000/api/optimize_model_params/9', json=json).text)

### 8.2 Неправильная передача гиперпараметров

In [ ]:
params_to_optimaze = {'C': [1, 1, 1]}

json = {'X': X_train,
       'y': y_train,
       'metric': 'f1',
       'params_to_optimaze': params_to_optimaze}


requests.get('http://127.0.0.1:5000/api/optimize_model_params/1', json=json).text

# 9. fit

### 9.1 Correct

In [ ]:
print(requests.put('http://127.0.0.1:5000/api/fit/2', json={'X': X_train, 'y': y_train}).text)

In [ ]:
print(requests.put('http://127.0.0.1:5000/api/fit/3', json={'X': Xm_train, 'y': ym_train}).text)

In [ ]:
print(requests.get('http://127.0.0.1:5000/api/results', json={'task_id': '7d8dbd8a-f318-4fae-b07d-a85836b35e1b'}).text)

# 10. predict

### 10.1 Correct

In [ ]:
print(requests.put('http://127.0.0.1:5000/api/predict/3', json={'X': Xm_test}).text)

In [ ]:
preds = requests.get('http://127.0.0.1:5000/api/results', json={'task_id': 'd4658675-ab09-4066-a6f9-f17ffd5b3027'}).json()

### 10.2 Incorrect

In [ ]:
print(requests.put('http://127.0.0.1:5000/api/predict/4', json={'X': X_test}).text)

# 11. predict_proba

### 11.1 Correct

In [ ]:
print(requests.put('http://127.0.0.1:5000/api/predict_proba/2', json={'X': X_test}).text)

In [ ]:
preds_proba = requests.get('http://127.0.0.1:5000/api/results', 
                           json={'task_id': '9263861a-453d-4bf6-ad2e-0cb930a2a645'}).json()

# 12. получение метрик качеств

### 12.1 Correct

In [ ]:
json = {'y_true': y_test, 
        'X': X_test, 
        'plot':True}

print(requests.get('http://127.0.0.1:5000/api/get_scores/2', json=json).text)

In [ ]:
print(requests.get('http://127.0.0.1:5000/api/results', json={'task_id': '4ecba3ff-e280-4203-907b-573a382aa9b4'}).text)

In [ ]:
json = {'y_true': ym_test, 
        'X': Xm_test}

print(requests.get('http://127.0.0.1:5000/api/get_scores/3', json=json).text)

In [ ]:
print(requests.get('http://127.0.0.1:5000/api/results', json={'task_id': '04a366cf-61ee-4d57-9cba-244cf9e926b3'}).text)